# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

- Implemented an interactive Gradio dashboard to accept user skills and a target Greenhouse API URL.
- Added automated tool-calling so the LLM can dynamically fetch and analyze real-time job listings.
- Enabled real-time response streaming and the ability to switch between different AI models.

In [ ]:
import os
import json
import requests
import gradio as gr
from bs4 import BeautifulSoup
from openai import OpenAI
from dotenv import load_dotenv

In [ ]:
load_dotenv(override=True)
api_key = os.getenv('OPENROUTER_API_KEY')

OPEN_ROUTER_BASE_URL = "https://openrouter.ai/api/v1"
client = OpenAI(api_key=api_key, base_url=OPEN_ROUTER_BASE_URL)

In [ ]:
# --- Define the Tool (Our Greenhouse API Fetcher) ---
def fetch_jobs_from_api(api_url, max_jobs=25):
    """Fetches job listings from the Greenhouse API using the full URL."""
     
    print(f"\n[Tool Execution] Fetching jobs from {api_url}...")
    
    try:
        response = requests.get(api_url)
        response.raise_for_status() 
        data = response.json()
        
        simplified_jobs = []
        for job in data.get('jobs', [])[:max_jobs]:
            clean_text = BeautifulSoup(job.get("content", ""), "html.parser").get_text(separator=" ", strip=True)
            simplified_jobs.append({
                "title": job.get("title"),
                "location": job.get("location", {}).get("name"),
                "url": job.get("absolute_url"),
                "description": clean_text[:1000] 
            })
        print(f"[Tool Execution] Successfully fetched {len(simplified_jobs)} jobs.")
        return json.dumps(simplified_jobs)
    
    except Exception as e:
        return json.dumps({"error": f"Failed to fetch jobs: {str(e)}"})

In [ ]:
# Describe the tool so the LLM knows how to use it
tools = [{
    "type": "function",
    "function": {
        "name": "fetch_jobs_from_api",
        "description": "Fetches a list of current open jobs and their descriptions from a provided Greenhouse API URL.",
        "parameters": {
            "type": "object",
            "properties": {
                "api_url": {
                    "type": "string",
                    "description": "The full Greenhouse API URL. e.g., 'https://boards-api.greenhouse.io/v1/boards/doordashusa/jobs?content=true'"
                }
            },
            "required": ["api_url"]
        }
    }
}]

In [ ]:
# --- Matchmaking Logic with Streaming and Tool Handling ---
def find_job_matches(user_skills, model_choice, custom_api_url):
    
    # Validation checks
    if not custom_api_url or custom_api_url.strip() == "":
        yield " **Missing URL:** To get started, please paste a Greenhouse API URL into the designated text box."
        return 
        
    if not user_skills or user_skills.strip() == "":
        yield " **Missing Skills:** Please enter your skills or job preferences so I can match you correctly!"
        return

    yield "Scanning job board and analyzing matches..."
    
    # Tell the LLM to use the provided URL
    url_instruction = f"You MUST use this exact URL for your tool to search for jobs: '{custom_api_url}'."

    system_prompt = {
        "role": "system", 
        "content": f"You are an expert technical career advisor and matchmaker. "
                   f"When a user shares their skills, use your 'fetch_jobs_from_api' tool to fetch open roles. "
                   f"{url_instruction} "
                   f"Read the job descriptions carefully, pick the best matches for their specific profile, and explain to the user WHY they are a good fit. "
                   f"Always include the URL so they can apply. Format your response beautifully with markdown."
    }
    
    # Construct the message array (No history needed!)
    messages = [
        system_prompt,
        {"role": "user", "content": user_skills}
    ]

    # Initial LLM call to decide if it needs the tool
    response = client.chat.completions.create(
        model=model_choice,
        messages=messages,
        tools=tools
    )
    
    assistant_message = response.choices[0].message

    # Handle Tool Calling
    if assistant_message.tool_calls:
        messages.append(assistant_message)
        
        for tool_call in assistant_message.tool_calls:
            if tool_call.function.name == "fetch_jobs_from_api":
                args = json.loads(tool_call.function.arguments)
                api_url = args.get("api_url")
                
                # Execute the tool
                tool_result = fetch_jobs_from_api(api_url=api_url)
                
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": tool_call.function.name,
                    "content": tool_result
                })
                
        # Stream the final response back to the UI
        stream_response = client.chat.completions.create(
            model=model_choice,
            messages=messages,
            stream=True 
        )
            
        partial_text = ""
        for chunk in stream_response:
            if chunk.choices[0].delta.content is not None:
                partial_text += chunk.choices[0].delta.content
                yield partial_text 
    else:
        # Fallback just in case the LLM decides not to use the tool
        yield assistant_message.content

In [ ]:
# --- Custom Gradio Dashboard ---
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# AI Career Matchmaker")
    gr.Markdown("Describe your skills, paste a company's Greenhouse API URL, and let the AI find the perfect role for you based on actual job descriptions.")
    
    with gr.Row():
        # Left Column for Inputs
        with gr.Column(scale=1):
            user_skills_input = gr.Textbox(
                label="Your Skills & Preferences", 
                placeholder="e.g., I am a Python developer with 3 years of experience in data engineering and AWS...",
                lines=5,
                interactive=True
            )
            api_url_input = gr.Textbox(
                label="Greenhouse API URL",
                placeholder="e.g., https://boards-api.greenhouse.io/v1/boards/doordashusa/jobs?content=true",
                lines=2,
                interactive=True
            )
            model_dropdown = gr.Dropdown(
                choices=["openai/gpt-4o-mini", "anthropic/claude-3-haiku", "meta-llama/llama-3.1-8b-instruct"],
                value="openai/gpt-4o-mini",
                label="Select AI Model"
            )
            submit_btn = gr.Button("Find My Matches", variant="primary")
            
        # Right Column for Outputs
        with gr.Column(scale=2):
            output_markdown = gr.Markdown(label="Job Matches", value="*Your matches will appear here...*")

    # --- Helper Functions for UI State ---
    def disable_btn():
        # Changes the button text and makes it unclickable
        return gr.update(value="Processing... ⏳", interactive=False)
        
    def enable_btn():
        # Changes the button text back and makes it clickable again
        return gr.update(value="Find My Matches", interactive=True)

    # --- Wire the button with Chained Events ---
    submit_btn.click(
        fn=disable_btn,
        inputs=[],
        outputs=[submit_btn]
    ).then(
        fn=find_job_matches,
        inputs=[user_skills_input, model_dropdown, api_url_input],
        outputs=[output_markdown]
    ).then(
        fn=enable_btn,
        inputs=[],
        outputs=[submit_btn]
    )

In [ ]:
demo.launch()